In [1]:
import sympy as sp
import numpy as np

import re
from sympy import symbols, Symbol, init_printing

from IPython.display import Markdown, Latex

from scipy.special import comb

# 1) Symbol を継承して _latex をオーバーライド
class DoubleIndexSymbol(Symbol):
    def _latex(self, printer):
        m = re.fullmatch(r'([A-Za-z]+)_(\d+)_(\d+)__(\d+)', self.name)
        if m:
            base, i, j, k = m.groups()
            return f"{base}_{{{i},{j}}}^{k}"
        return self.name

# 2) MathJax 表示を有効化
init_printing(use_latex='mathjax')

In [2]:
n = 6
d = 4

file_path = f'data/n{n}-d{d}.dat-s'

In [3]:
C = [
    "000", "111"
]

# C = [
#     "1000", "0100", "0010", "1110",
#     "0001", "1101", "1011", "0111",
# ]

# C = ["10", "01"]

C = [
    "000000", "001111", "111100", "110011"
]

In [4]:
def encode_variable(n, t, i, j):
    return i + (n + 1) * j + (n + 1) * (n + 1) * t + 1

def decode_variable_index(var_idx):
    var_idx -= 1
    i = var_idx % (n + 1)
    var_idx //= (n + 1)
    j = var_idx % (n + 1)
    var_idx //= (n + 1)
    t = var_idx % (n + 1)
    return i, j, t

def get_variable_symbol(var_idx):
    if var_idx == 0:
        return 'x_0'
    i, j, t = decode_variable_index(var_idx)
    return f"x_{i}_{j}__{t}"

In [5]:
## validation

for i in range(n + 1):
    for j in range(n + 1):
        for t in range(n + 1):
            var_idx = encode_variable(n, t, i, j)
            assert (i, j, t) == decode_variable_index(var_idx)

for idx in range(1, (n + 1) ** 3):
    i, j, t = decode_variable_index(idx)
    assert idx == encode_variable(n, t, i, j)

In [6]:
with open(file_path) as f:
    line = f.readline()
    num_variables = int(line.split()[0])

    var_str = " ".join([get_variable_symbol(i) for i in range(num_variables + 1)])
    variables = sp.symbols(var_str, cls=DoubleIndexSymbol)

    line = f.readline()
    nblock = int(line.split()[0])

    line = f.readline()
    block_sizes = map(int, line.split())

    line = f.readline()
    objective = map(int, line.split())
    
    line = f.readline()

    matrices = [
        sp.Matrix(abs(s), abs(s), lambda i,j: 0) for s in block_sizes
    ]

    while line:
        var_idx, block_idx, row, col, val = map(int, line.split())
        if var_idx == 0:
            matrices[block_idx - 1][row - 1, col - 1] += (-1) * val
        else:
            matrices[block_idx - 1][row - 1, col - 1] += val * variables[var_idx]
        line = f.readline()

In [7]:
# symmetrize
for mat in matrices:
    rows, cols = mat.shape
    for row in range(rows):
        for col in range(cols):
            if row <= col:
                continue
            mat[row, col] = mat[col, row]

In [8]:
variables

(x₀, x⁰₀ ₀, x⁰₁ ₀, x⁰₂ ₀, x⁰₃ ₀, x⁰₄ ₀, x⁰₅ ₀, x⁰₆ ₀, x⁰₀ ₁, x⁰₁ ₁, x⁰₂ ₁, x⁰₃ ↪

↪  ₁, x⁰₄ ₁, x⁰₅ ₁, x⁰₆ ₁, x⁰₀ ₂, x⁰₁ ₂, x⁰₂ ₂, x⁰₃ ₂, x⁰₄ ₂, x⁰₅ ₂, x⁰₆ ₂, x⁰ ↪

↪ ₀ ₃, x⁰₁ ₃, x⁰₂ ₃, x⁰₃ ₃, x⁰₄ ₃, x⁰₅ ₃, x⁰₆ ₃, x⁰₀ ₄, x⁰₁ ₄, x⁰₂ ₄, x⁰₃ ₄, x ↪

↪ ⁰₄ ₄, x⁰₅ ₄, x⁰₆ ₄, x⁰₀ ₅, x⁰₁ ₅, x⁰₂ ₅, x⁰₃ ₅, x⁰₄ ₅, x⁰₅ ₅, x⁰₆ ₅, x⁰₀ ₆,  ↪

↪ x⁰₁ ₆, x⁰₂ ₆, x⁰₃ ₆, x⁰₄ ₆, x⁰₅ ₆, x⁰₆ ₆, x¹₀ ₀, x¹₁ ₀, x¹₂ ₀, x¹₃ ₀, x¹₄ ₀, ↪

↪  x¹₅ ₀, x¹₆ ₀, x¹₀ ₁, x¹₁ ₁, x¹₂ ₁, x¹₃ ₁, x¹₄ ₁, x¹₅ ₁, x¹₆ ₁, x¹₀ ₂, x¹₁ ₂ ↪

↪ , x¹₂ ₂, x¹₃ ₂, x¹₄ ₂, x¹₅ ₂, x¹₆ ₂, x¹₀ ₃, x¹₁ ₃, x¹₂ ₃, x¹₃ ₃, x¹₄ ₃, x¹₅  ↪

↪ ₃, x¹₆ ₃, x¹₀ ₄, x¹₁ ₄, x¹₂ ₄, x¹₃ ₄, x¹₄ ₄, x¹₅ ₄, x¹₆ ₄, x¹₀ ₅, x¹₁ ₅, x¹₂ ↪

↪  ₅, x¹₃ ₅, x¹₄ ₅, x¹₅ ₅, x¹₆ ₅, x¹₀ ₆, x¹₁ ₆, x¹₂ ₆, x¹₃ ₆, x¹₄ ₆, x¹₅ ₆, x¹ ↪

↪ ₆ ₆, x²₀ ₀, x²₁ ₀, x²₂ ₀, x²₃ ₀, x²₄ ₀, x²₅ ₀, x²₆ ₀, x²₀ ₁, x²₁ ₁, x²₂ ₁, x ↪

↪ ²₃ ₁, x²₄ ₁, x²₅ ₁, x²₆ ₁, x²₀ ₂, x²₁ ₂, x²₂ ₂, x²₃ ₂, x²₄ ₂, x²₅ ₂, x²₆ ₂,  ↪

↪ x²₀ ₃, x²₁ ₃, x²₂ ₃, x²₃ ₃, x²₄ ₃, x²₅ ₃, x²₆ ₃, x²₀ ₄, x²₁ ₄, x²₂ ₄, x²₃ ₄, ↪

↪  x²₄ ₄, x²₅ ₄,

In [9]:
block_idx = 1

matrices[block_idx - 1]

⎡ x⁰₀ ₀          6⋅x⁰₁ ₀                   15⋅x⁰₂ ₀                            ↪
⎢                                                                              ↪
⎢6⋅x⁰₁ ₀   6⋅x⁰₀ ₀ + 30⋅x⁰₂ ₀         30⋅x⁰₁ ₀ + 60⋅x⁰₃ ₀                    6 ↪
⎢                                                                              ↪
⎢15⋅x⁰₂ ₀  30⋅x⁰₁ ₀ + 60⋅x⁰₃ ₀  15⋅x⁰₀ ₀ + 120⋅x⁰₂ ₀ + 90⋅x⁰₄ ₀        60⋅x⁰₁  ↪
⎢                                                                              ↪
⎢20⋅x⁰₃ ₀  60⋅x⁰₂ ₀ + 60⋅x⁰₄ ₀  60⋅x⁰₁ ₀ + 180⋅x⁰₃ ₀ + 60⋅x⁰₅ ₀  20⋅x⁰₀ ₀ + 18 ↪
⎢                                                                              ↪
⎢15⋅x⁰₄ ₀  60⋅x⁰₃ ₀ + 30⋅x⁰₅ ₀  90⋅x⁰₂ ₀ + 120⋅x⁰₄ ₀ + 15⋅x⁰₆ ₀        60⋅x⁰₁  ↪
⎢                                                                              ↪
⎢6⋅x⁰₅ ₀   30⋅x⁰₄ ₀ + 6⋅x⁰₆ ₀         60⋅x⁰₃ ₀ + 30⋅x⁰₅ ₀                    6 ↪
⎢                                                                              ↪
⎣ x⁰₆ ₀          6⋅x⁰₅ ₀    

In [10]:
def calc_sym_diff(x, y, z):
    """
    * x, y, z \in C \subset {0, 1}^n
    * i=|x \bigtriangleup y|, j=|x \bigtriangleup z|, t=|(x \bigtriangleup y) \cap (x \bigtriangleup z)|
    """
    i, j, t = 0, 0, 0
    for idx in range(len(x)):
        if x[idx] != y[idx]:
            i += 1
        if x[idx] != z[idx]:
            j += 1
        if x[idx] != y[idx] and x[idx] != z[idx]:
            t += 1
    return i, j, t

def lambda_to_x(lamb, code_size, t, i, j):
    denom = (code_size * comb(n, i-t, exact=True) * comb(n-i+t,j-t, exact=True) * comb(n-i-j+2*t, t, exact=True))
    if denom == 0:
        return 0
    return lamb / denom

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
/var/folders/6k/nrp954t51fq50vv_ch4k2wc00000gn/T/ipykernel_12181/1651595868.py:2: SyntaxWarning: invalid escape sequence '\i'
  """


In [11]:
effs_lambda = [0.0 for _ in range(num_variables + 1)]

indices = []
for x in C:
    for y in C:
        for z in C:
            i, j, t = calc_sym_diff(x, y, z)
            var_idx = encode_variable(n, t, i, j)
            indices.append(var_idx)
            effs_lambda[var_idx] += 1

effs = [0.0 for _ in range(num_variables + 1)]
for i in range(n + 1):
    for j in range(n + 1):
        for t in range(n + 1):
            var_idx = encode_variable(n, t, i, j)
            effs[var_idx] = lambda_to_x(effs_lambda[var_idx], len(C), t, i, j)

for i in range(1, num_variables + 1):
    if effs[i] == 0:
        continue
    display(f"{variables[i]}={effs[i]}")

effs

'x_0_0__0=1.0'

'x_4_0__0=0.2'

'x_0_4__0=0.2'

'x_4_4__2=0.06666666666666667'

'x_4_4__4=0.2'

[0.0, 1.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, ↪

↪  0.0, 0.0, 0.0, 0.0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0.2, 0.0, 0.0, 0, 0, ↪

↪  0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0.0, 0.0, ↪

↪  0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0.0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0. ↪

↪ 0, 0.0, 0.0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0.0, 0.0, 0.0666666666666666 ↪

↪ 7, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0,  ↪

↪ 0, 0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0.2, 0.0, 0.0, 0, 0, 0, 0, 0.0, 0.0, 0, 0, 0, 0, 0, 0.0, 0,  ↪

↪ 0, 0, 0, 0, 0,

In [12]:
subs_map = {}
for i in range(1, num_variables + 1):
    subs_map[variables[i]] = effs[i]

print(subs_map)

subs_matrices = []
for mat in matrices:
    subs_matrices.append(mat.subs(subs_map))

subs_matrices[0]

{x_0_0__0: 1.0, x_1_0__0: 0.0, x_2_0__0: 0.0, x_3_0__0: 0.0, x_4_0__0: 0.2, x_5_0__0: 0.0, x_6_0__0: 0.0, x_0_1__0: 0.0, x_1_1__0: 0.0, x_2_1__0: 0.0, x_3_1__0: 0.0, x_4_1__0: 0.0, x_5_1__0: 0.0, x_6_1__0: 0, x_0_2__0: 0.0, x_1_2__0: 0.0, x_2_2__0: 0.0, x_3_2__0: 0.0, x_4_2__0: 0.0, x_5_2__0: 0, x_6_2__0: 0, x_0_3__0: 0.0, x_1_3__0: 0.0, x_2_3__0: 0.0, x_3_3__0: 0.0, x_4_3__0: 0, x_5_3__0: 0, x_6_3__0: 0, x_0_4__0: 0.2, x_1_4__0: 0.0, x_2_4__0: 0.0, x_3_4__0: 0, x_4_4__0: 0, x_5_4__0: 0, x_6_4__0: 0, x_0_5__0: 0.0, x_1_5__0: 0.0, x_2_5__0: 0, x_3_5__0: 0, x_4_5__0: 0, x_5_5__0: 0, x_6_5__0: 0, x_0_6__0: 0.0, x_1_6__0: 0, x_2_6__0: 0, x_3_6__0: 0, x_4_6__0: 0, x_5_6__0: 0, x_6_6__0: 0, x_0_0__1: 0, x_1_0__1: 0, x_2_0__1: 0, x_3_0__1: 0, x_4_0__1: 0, x_5_0__1: 0, x_6_0__1: 0, x_0_1__1: 0, x_1_1__1: 0.0, x_2_1__1: 0.0, x_3_1__1: 0.0, x_4_1__1: 0.0, x_5_1__1: 0.0, x_6_1__1: 0.0, x_0_2__1: 0, x_1_2__1: 0.0, x_2_2__1: 0.0, x_3_2__1: 0.0, x_4_2__1: 0.0, x_5_2__1: 0.0, x_6_2__1: 0, x_0_3__1: 0

⎡1.0   0     0     0    3.0    0    0.0⎤
⎢                                      ⎥
⎢ 0   6.0    0    12.0   0    6.0    0 ⎥
⎢                                      ⎥
⎢ 0    0    33.0   0    24.0   0    3.0⎥
⎢                                      ⎥
⎢ 0   12.0   0    56.0   0    12.0   0 ⎥
⎢                                      ⎥
⎢3.0   0    24.0   0    33.0   0     0 ⎥
⎢                                      ⎥
⎢ 0   6.0    0    12.0   0    6.0    0 ⎥
⎢                                      ⎥
⎣0.0   0    3.0    0     0     0    0.0⎦

In [13]:
find_not_psd = False

for block_idx, mat in enumerate(subs_matrices):
    eigenvals = mat.eigenvals()
    is_psd = all(ev >= 0 for ev in eigenvals.keys())

    if not is_psd:
        print(f'block [{block_idx + 1}] is not positive semedefinite!')
        print(f'\teigenvals = ', eigenvals)
        display(mat)
        display(matrices[block_idx])
        find_not_psd = True

if not find_not_psd:
    print("Correct!")

block [1] is not positive semedefinite!
	eigenvals =  {61.7848879788996: 1, 6.21511202110039: 1, 2.04938826921182e-65: 1, 57.1588578309577: 1, 9.95454029442599: 1, 0.562037318810617: 1, -0.675435444194350: 1}


⎡1.0   0     0     0    3.0    0    0.0⎤
⎢                                      ⎥
⎢ 0   6.0    0    12.0   0    6.0    0 ⎥
⎢                                      ⎥
⎢ 0    0    33.0   0    24.0   0    3.0⎥
⎢                                      ⎥
⎢ 0   12.0   0    56.0   0    12.0   0 ⎥
⎢                                      ⎥
⎢3.0   0    24.0   0    33.0   0     0 ⎥
⎢                                      ⎥
⎢ 0   6.0    0    12.0   0    6.0    0 ⎥
⎢                                      ⎥
⎣0.0   0    3.0    0     0     0    0.0⎦

⎡ x⁰₀ ₀          6⋅x⁰₁ ₀                   15⋅x⁰₂ ₀                            ↪
⎢                                                                              ↪
⎢6⋅x⁰₁ ₀   6⋅x⁰₀ ₀ + 30⋅x⁰₂ ₀         30⋅x⁰₁ ₀ + 60⋅x⁰₃ ₀                    6 ↪
⎢                                                                              ↪
⎢15⋅x⁰₂ ₀  30⋅x⁰₁ ₀ + 60⋅x⁰₃ ₀  15⋅x⁰₀ ₀ + 120⋅x⁰₂ ₀ + 90⋅x⁰₄ ₀        60⋅x⁰₁  ↪
⎢                                                                              ↪
⎢20⋅x⁰₃ ₀  60⋅x⁰₂ ₀ + 60⋅x⁰₄ ₀  60⋅x⁰₁ ₀ + 180⋅x⁰₃ ₀ + 60⋅x⁰₅ ₀  20⋅x⁰₀ ₀ + 18 ↪
⎢                                                                              ↪
⎢15⋅x⁰₄ ₀  60⋅x⁰₃ ₀ + 30⋅x⁰₅ ₀  90⋅x⁰₂ ₀ + 120⋅x⁰₄ ₀ + 15⋅x⁰₆ ₀        60⋅x⁰₁  ↪
⎢                                                                              ↪
⎢6⋅x⁰₅ ₀   30⋅x⁰₄ ₀ + 6⋅x⁰₆ ₀         60⋅x⁰₃ ₀ + 30⋅x⁰₅ ₀                    6 ↪
⎢                                                                              ↪
⎣ x⁰₆ ₀          6⋅x⁰₅ ₀    

block [2] is not positive semedefinite!
	eigenvals =  {3.20000000000000: 1, -4.44089209850063e-16: 1, 3.60000000000000: 1, 0.800000000000000: 1, -2.59052039079203e-16: 1}


⎡1.0    0    -1.2   0    0.2 ⎤
⎢                            ⎥
⎢ 0    1.6    0    -1.6   0  ⎥
⎢                            ⎥
⎢-1.2   0    2.4    0    -1.2⎥
⎢                            ⎥
⎢ 0    -1.6   0    1.6    0  ⎥
⎢                            ⎥
⎣0.2    0    -1.2   0    1.0 ⎦

⎡  x⁰₀ ₀ - x⁰₂ ₀         4⋅x⁰₁ ₀ - 4⋅x⁰₃ ₀                   6⋅x⁰₂ ₀ - 6⋅x⁰₄ ₀ ↪
⎢                                                                              ↪
⎢4⋅x⁰₁ ₀ - 4⋅x⁰₃ ₀  4⋅x⁰₀ ₀ + 8⋅x⁰₂ ₀ - 12⋅x⁰₄ ₀            12⋅x⁰₁ ₀ - 12⋅x⁰₅  ↪
⎢                                                                              ↪
⎢6⋅x⁰₂ ₀ - 6⋅x⁰₄ ₀      12⋅x⁰₁ ₀ - 12⋅x⁰₅ ₀       6⋅x⁰₀ ₀ + 18⋅x⁰₂ ₀ - 18⋅x⁰₄  ↪
⎢                                                                              ↪
⎢4⋅x⁰₃ ₀ - 4⋅x⁰₅ ₀  12⋅x⁰₂ ₀ - 8⋅x⁰₄ ₀ - 4⋅x⁰₆ ₀            12⋅x⁰₁ ₀ - 12⋅x⁰₅  ↪
⎢                                                                              ↪
⎣  x⁰₄ ₀ - x⁰₆ ₀         4⋅x⁰₃ ₀ - 4⋅x⁰₅ ₀                   6⋅x⁰₂ ₀ - 6⋅x⁰₄ ₀ ↪

↪                   4⋅x⁰₃ ₀ - 4⋅x⁰₅ ₀          x⁰₄ ₀ - x⁰₆ ₀  ⎤
↪                                                             ⎥
↪ ₀            12⋅x⁰₂ ₀ - 8⋅x⁰₄ ₀ - 4⋅x⁰₆ ₀  4⋅x⁰₃ ₀ - 4⋅x⁰₅ ₀⎥
↪                                                             ⎥
↪ ₀ - 6⋅x⁰₆ ₀ 

In [14]:
objective = [val for val in objective]
objective

[-1, -6, -15, -20, -15, -6, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ↪

↪ , 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ↪

↪  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  ↪

↪ 0, 0, 0, 0, 0,

In [15]:
int(np.dot(effs[1:], objective))

-4